# Gene ontology analysis of spleen samples

In [ ]:
import numpy as np
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.legend import Legend
import matplotlib.colors as colors
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import matplotlib.patches as patches
import pandas as pd
import scipy
import scanpy as sc
import anndata as ad

import random

import pydeseq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

import gseapy as gp
from gseapy.plot import gseaplot

eps= 10.**(-300.)

In [ ]:
sc.set_figure_params(scanpy=True, dpi=600, dpi_save=1200, frameon=True, vector_friendly=False, fontsize=14,
                         figsize=(9,8),  format='pdf', facecolor=None, transparent=False, ipython_format='png2x')

#### GOA based on Sanbomics tutorial https://www.youtube.com/watch?v=ONiWugVEf2s

#!pip install goatools

### biomaRt not used in this notebook.
###!pip install biomart

###!pip install adjustText

###!pip install gprofiler

In [ ]:
#!mv /Users/oipulk/Downloads/gene_result.txt .
#!python /Users/oipulk/miniconda3/envs/py3/bin/ncbi_gene_results_to_python.py -o genes_ncbi_mus_musculus_proteincoding.py gene_result.txt

In [ ]:
adata = sc.read("maranou_032024_spleen_annotated.h5ad")
dc_types = ["Lymphoid-resident cDC1","CD8(low) cDC1","CD8- CCR2+ cDC1","CD8- CCR2- cDC1", "CCR7+ DC1", 
                        "Relb(int.) cDC2","Migratory cDC2","WDFY4+ cDC2","Relb(low) cDC2","pDC","Mixed DC","Undefined DC"]
adata_subset = adata[adata.obs['cell_type_high_res'].isin(dc_types)].copy()

In [ ]:
adata_subset.var_names[adata_subset.var['highly_variable']]

In [ ]:
hvgs = set(adata_subset.var_names[adata_subset.var['highly_variable']])
len(hvgs)

In [ ]:
min_expr_threshold = 0.1  # adjust based on your normalization
dc_expressed = set(adata_subset.var_names[np.asarray(adata_subset.X.mean(axis=0))[0] >= min_expr_threshold])
len(dc_expressed)

In [ ]:
len(hvgs.intersection(dc_expressed))

In [ ]:
from genes_ncbi_mus_musculus_proteincoding import GENEID2NT as GeneID2nt_mus
from goatools.base import download_go_basic_obo, download_ncbi_associations
from goatools.obo_parser import GODag
from goatools.anno.genetogo_reader import Gene2GoReader
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS
import pandas as pd

from genes_ncbi_mus_musculus_proteincoding import GENEID2NT as GeneID2nt_mus

obo_fname = download_go_basic_obo()
fin_gene2go = download_ncbi_associations()
obodag = GODag("go-basic.obo")

# Define map and inverse map between gene symbol and ID
mapper = {}

for key in GeneID2nt_mus:
    mapper[GeneID2nt_mus[key].Symbol] = GeneID2nt_mus[key].GeneID
    
inv_map = {v: k for k, v in mapper.items()}


# Read NCBI's gene2go. Store annotations in a list of namedtuples
objanno = Gene2GoReader(fin_gene2go, taxids=[10090])
# Get namespace2association where:
#    namespace is:
#        BP: biological_process               
#        MF: molecular_function
#        CC: cellular_component
#    assocation is a dict:
#        key: NCBI GeneID
#        value: A set of GO IDs associated with that gene
ns2assoc = objanno.get_ns2assc()

### To narrow down the set of background genes, we can take only the highly variable ones in our data
# expressed_genes = set(adata.var_names[adata.var['highly_variable']])
# # Convert gene symbols to NCBI IDs
# expressed_gene_ids = {mapper[gene] for gene in expressed_genes 
#                      if gene in mapper}

# Or get genes with meaningful expression
min_expr_threshold = 0.1  # adjust based on your normalization
dc_genes =set(adata_subset.var_names[np.asarray(adata_subset.X.mean(axis=0))[0] >= min_expr_threshold])

# Convert to NCBI IDs
dc_gene_ids = {mapper[gene] for gene in dc_genes 
               if gene in mapper}

goeaobj = GOEnrichmentStudyNS(
        # dc_gene_ids,
        GeneID2nt_mus.keys(), # List of mouse protein-coding genes
        ns2assoc, # geneid/GO associations
        obodag, # Ontologies
        propagate_counts = True,
        alpha = 0.05, # default significance cut-off
        methods = ['fdr_bh']) # default multipletest correction method

GO_items = []

temp = goeaobj.ns2objgoea['BP'].assoc
for item in temp:
    GO_items += temp[item]
    
temp = goeaobj.ns2objgoea['CC'].assoc
for item in temp:
    GO_items += temp[item]
    
temp = goeaobj.ns2objgoea['MF'].assoc
for item in temp:
    GO_items += temp[item]


# # Define immune-relevant root terms and keywords
IMMUNE_ROOT_TERMS = {

    # #All GO terms 
    # 'GO:0008150', # BP
    # 'GO:0003674', # MF
    # 'GO:0005575', # CC
 
    # Core immune processes
    'GO:0002376',  # immune system process
    'GO:0050896',  # response to stimulus
    'GO:0006955',  # response to stimulus - immune response
    'GO:0009987',  # Cellular process
    'GO:0050789', # Regulation of biological process
    'GO:0008219', # Cell death
    'GO:0045321',  # leukocyte activation
    'GO:0005515', # cytokine binding
    
    'GO:0002520',  # immune system development
    'GO:0002682',  # regulation of immune system process
    'GO:0006954',  # inflammatory response
    'GO:0002250',  # adaptive immune response
    'GO:0045087',  # innate immune response
    'GO:0030333',  # antigen processing and presentation
    
    # # DC-specific processes
    'GO:0002396',  # MHC protein complex assembly
    
    'GO:0071404',  # cellular response to low-density lipoprotein particle stimulus
    'GO:0071396',  # cellular response to lipid
    'GO:0019886',  # antigen processing and presentation via MHC class II
    'GO:0002606',  # positive regulation of dendritic cell differentiation
    'GO:0002819',  # regulation of adaptive immune response
    
    # # Tumor microenvironment relevant
    'GO:0002837',  # regulation of immune response to tumor cell
    'GO:0097530',  # granulocyte migration
    'GO:0002548',  # monocyte chemotaxis
    'GO:0042102',  # positive regulation of T cell proliferation
    'GO:0042110',  # T cell activation
    'GO:0050863',  # regulation of T cell activation
    
    # # Cellular processes relevant to DCs
    'GO:0016192',  # vesicle-mediated transport
    'GO:0006897',  # endocytosis
    'GO:0034341',  # response to interferon-gamma
    'GO:0071347',  # cellular response to interleukin-1
    'GO:0035872',  # nucleotide-binding domain, leucine rich repeat containing receptor signaling pathway
    
    # # Additional immune processes
    'GO:0002764',  # immune response-regulating signaling pathway
    'GO:0050776',  # regulation of immune response
    'GO:0002684',  # positive regulation of immune system process
    'GO:0002683',  # negative regulation of immune system process
    'GO:0002521',  # leukocyte differentiation
    'GO:0002366',  # leukocyte activation involved in immune response
    'GO:0002460',  # adaptive immune response based on somatic recombination
    'GO:0002757',  # immune response-activating signal transduction
}

IMMUNE_KEYWORDS = {
    # Core immune terms
    'immune', 'lymphocyte', 'leukocyte', 'cytokine',
    'chemokine', 'inflammation', 'inflammatory', 'antigen', 'antibody',
    'tcr', 'bcr', 'mhc', 'nfkb', 'toll', 'interferon',
    'interleukin', 'tnf', 'immunoglobulin',
    
    # DC-specific terms
    'dendritic', 'cd11c', 'cd83', 'cd86', 'cd40',
    'batf3', 'zbtb46', 'xcr1', 'clec9a', 'cd103',
    'sirpa', 'cd207', 'cd1c', 'cd141',
    
    # Antigen presentation and processing
    'tap', 'cathepsin', 'lamp', 'rab', 'snap',
    'vamp', 'h2-dm', 'h2-do', 'clip', 'ciita',
    
    # Signal transduction
    'stat', 'jak', 'traf', 'irf', 'syk',
    'mapk', 'pi3k', 'nf-kb','nf-kappab', 'tgf', 'notch',
    
    # Migration and adhesion
    'ccr', 'cxcr', 'integrin', 'selectin', 'vcam',
    'icam', 'mmp', 'itga', 'itgb', 'migratory','migration',
    
    # Tumor microenvironment
    'pd-l1', 'cd80', 'ido', 'arg1', 'vegf',
    'tgfb', 'il10', 'stat3', 'hippo', 'ros',
    
    # Metabolism and stress
    'mtor', 'ampk', 'hif', 'ros', 'autophagy',
    'glycolysis', 'oxidative', 'mitochondrial',
    
    # Trafficking and processing
    'endosome', 'lysosome', 'golgi', 'vesicle',
    'phagosome', 'autophagy', 'endocytic',
    
    # Transcriptional regulation
    'transcription', 'chromatin', 'epigenetic',
    'methylation', 'acetylation', 'histone',
    
    # Additional immune terms
    'granzyme', 'perforin', 'complement',
    'inflammasome', 'tlr', 'nod', 'rig-i',
    'mda5', 'nlrp3', 'asc', 'caspase',
    
    # Cytokine signaling
    'il1', 'il2', 'il4', 'il6', 'il10',
    'il12', 'il17', 'il23', 'ifng', 'tnfa',
    
    # T cell related
    't cell', 'cd3', 'cd4', 'cd8', 'foxp3', 'rorc',
    'tbet', 'gata3', 'cd25', 'cd69',
    
    # B cell related
    'b cell', 'cd19', 'cd20', 'cd27', 'cd38', 'cd138',
    'baff', 'april', 'plasma cell',
    
    # Myeloid cell related
    'myeloid', 'cd14', 'cd16', 'cd64', 'cd68', 'cd163',
    'marco', 'mertk', 'trem2',
    
    # Additional trafficking terms
    'cxcl', 'ccl', 'sphingosine', 'ceramide',
    's1p', 'lpa', 'prostaglandin',

    # Additional Immune System Terms:
    'innate', 'adaptive', 'spleen', 
    'nk cell', 'natural killer',
    'myeloid', 'neutrophil', 'eosinophil', 'basophil',
    'ifna', 'ifnb',
    
    #Cancer/Oncology Related Terms:
    'neoplasm', 'oncogene', 'tumor suppressor',
    'metastasis', 'angiogenesis', 'invasion',
    'warburg', 'emt', 'mesenchymal',
    'p53', 'myc', 'ras', 'brca', 
    
    #Cancer Hallmark Terms:
    'telomerase', 'senescence', 'apoptosis'
    'stemness', 'cancer stem cell'
    'dna repair', 'genome instability'
    'immortalization', 'proliferation', 
    
    #Tumor Microenvironment Additions:
    'mdsc','cam','caf', 'suppressor', 
    'cancer', 'macrophage', 'fibroblast',
    'hypoxia', 'angiogenesis', 
    
    #Immune Checkpoint Terms:
    'checkpoint', 'ctla4', 'lag3', 'tim3', 'vista',
    'pd1', 'pdcd1', 'btla',
    'icos', 'gitr', 'ox40', 
    
    #Signaling Pathway Completeness:
    'wnt', 'hedgehog', 'hippo',
    'akt', 'pten', 'pi3k',
    'smad', 'bmp', 'jun', 'fos' 

}


def get_immune_relevant_children(obodag, root_terms=IMMUNE_ROOT_TERMS):
    """Get all children of immune-relevant root terms"""
    relevant_terms = set()
    for term in root_terms:
        if term in obodag:
            relevant_terms.add(term)
            relevant_terms.update(obodag[term].get_all_children())
    return relevant_terms

def is_immune_relevant(go_term, immune_terms, immune_keywords=IMMUNE_KEYWORDS):
    """Check if a GO term is immune-relevant"""
    if go_term.id in immune_terms:
        return True
    return any(keyword in go_term.name.lower() for keyword in immune_keywords)

# Your existing setup code remains the same until the go_it function
def go_it(test_genes, immune_filter=True):
    print(f'input genes: {len(test_genes)}')
    
    mapped_genes = []
    for gene in test_genes:
        try:
            mapped_genes.append(mapper[gene])
        except:
            pass
    print(f'mapped genes: {len(mapped_genes)}')
    
    # Get immune-relevant terms
    immune_terms = get_immune_relevant_children(obodag)
    
    goea_results_all = goeaobj.run_study(mapped_genes)
    goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]
    
    # Filter for immune-relevant terms if requested
    if immune_filter:
        goea_results_sig = [r for r in goea_results_sig 
                           if is_immune_relevant(r.goterm, immune_terms)]
    
    GO = pd.DataFrame(
        list(map(
            lambda x: [
                x.GO, 
                x.goterm.name, 
                x.goterm.namespace,
                x.p_uncorrected,
                x.p_fdr_bh,
                x.ratio_in_study[0],
                x.ratio_in_study[1],
                GO_items.count(x.GO),
                list(map(lambda y: inv_map[y], x.study_items)),
            ],
            goea_results_sig
        )),
        columns=['GO', 'term', 'class', 'p', 'p_corr', 'n_genes',
                'n_study', 'n_go', 'study_genes']
    )
    
    # Split by class and handle each separately
    go_classes = []
    for class_name in GO['class'].unique():
        class_df = GO[GO['class'] == class_name].copy()
        if len(class_df) > 0:  # Only process if class has any terms
            class_df['immune_relevance'] = class_df.apply(
                lambda row: sum(1 for kw in IMMUNE_KEYWORDS 
                if kw in row['term'].lower()) / len(IMMUNE_KEYWORDS),
                axis=1
            )
            # Filter and sort within each class
            class_df = class_df[class_df.n_genes > 1].sort_values(
                by=['immune_relevance', 'p_corr'],
                ascending=[False, True]
            )
            go_classes.append(class_df)

    # Combine results if any exist
    if go_classes:
        GO = pd.concat(go_classes, axis=0)
    else:
        # Create empty DataFrame with same structure if no results at all
        GO = pd.DataFrame(columns=GO.columns)
        GO['immune_relevance'] = pd.Series(dtype=float)
    
    return GO


  


In [ ]:
immune_terms = get_immune_relevant_children(obodag)

In [ ]:
len(immune_terms)

In [ ]:
# Whether to use interaction DEA or not

interaction = True
# interaction = False


In [ ]:

if interaction:

    ## Aggregated DC with interaction DEA
    dc_markers = pd.read_csv('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/pseudobulk/spleen/deseq2_interaction_results/Spleen_kopat_vs_rest_interaction_degs_aggregatedDC.csv')
    dc_markers = dc_markers.rename(columns={"gene": "Gene"})

##################################################
else:
    
    ## Aggregated DC:
    dc_markers = pd.read_csv('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/pseudobulk/spleen/deseq2_pathogenic_results/Spleen_pat_kowt_degs_aggregatedDC.csv')
    dc_markers = dc_markers.rename(columns={"Unnamed: 0": "Gene"})

# Restrict the set of genes to high LFC
# dc_markers_up = dc_markers[dc_markers['log2FoldChange']>1.0]
# dc_markers_down = dc_markers[dc_markers['log2FoldChange']<-1.0]

dc_markers_up = dc_markers[dc_markers['log2FoldChange']>0.5]
dc_markers_down = dc_markers[dc_markers['log2FoldChange']<-0.5]

# Drop the KO gene if it is still in the set (not necessarily in interaction analysis)
cd74_up_idx = dc_markers_up[dc_markers_up['Gene']=='Cd74'].index
if len(cd74_up_idx)>0:
    dc_markers_up = dc_markers_up.drop(cd74_up_idx[0], axis=0)

cd74_down_idx = dc_markers_down[dc_markers_down['Gene']=='Cd74'].index
if len(cd74_down_idx)>0:
    dc_markers_down = dc_markers_down.drop(cd74_down_idx[0], axis=0)



In [ ]:
# Get all GO terms

immune_filter = True

# df_all = go_it(dc_markers.Gene.values, immune_filter=False)

# Get only immune-relevant terms
df_immune_up = go_it(dc_markers_up.Gene.values, immune_filter=immune_filter)
df_immune_down = go_it(dc_markers_down.Gene.values, immune_filter=immune_filter)

In [ ]:
df_immune_up['up_down'] = 'up'
df_immune_down['up_down'] = 'down'

df_immune_up['-logp_corr']= -np.log(eps+df_immune_up['p_corr'])
df_immune_up['per'] = (df_immune_up.n_genes/df_immune_up.n_go).astype(float)

df_immune_up['score'] =  np.log(df_immune_up['-logp_corr']) + np.log(df_immune_up['per'])
df_immune_up = df_immune_up.sort_values('-logp_corr')[::-1]


df_immune_down['-logp_corr']= -np.log(eps+df_immune_down['p_corr'])
df_immune_down['per'] = (df_immune_down.n_genes/df_immune_down.n_go).astype(float)

df_immune_down['score'] =  np.log(df_immune_down['-logp_corr']) + np.log(df_immune_down['per'])
df_immune_down = df_immune_down.sort_values('-logp_corr')[::-1]


In [ ]:
## Here you can choose between up and down regulated DEGs, or all of them
## For interaction DEA, use up-regulated only (only a few DR, inparticular Hba, Hbb)
upregulated_degs_only = False
downregulated_degs_only = False

if downregulated_degs_only|upregulated_degs_only:

    if downregulated_degs_only:
        df = df_immune_down.copy()
    
    if upregulated_degs_only:
        df = df_immune_up.copy()

else:

    df = pd.concat([df_immune_up, df_immune_down])

# Sort by p_corrected
df= df.sort_values('-logp_corr')[::-1]
# or by score
# df= df.sort_values('score')[::-1]


In [ ]:
### Save GO data
############

if interaction:
    
    if upregulated_degs_only:
        
        df.to_csv('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_aggreagatedDC_interaction_patKO_upregulated_degs.csv')

    else:
            
        df.to_csv('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_aggreagatedDC_interaction_patKO_all_degs.csv')



else:
    
    if upregulated_degs_only:

        df.to_csv('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_aggreagatedDC_KO_vs_WT_upregulated_degs.csv')

    else:

        df.to_csv('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_aggreagatedDC_KO_vs_WT_all_degs.csv')



In [ ]:

immune_relevance_filter = True

if immune_relevance_filter:
    print(df[df['immune_relevance']>0])

else: 
    print(df)

In [ ]:
list(df[df['immune_relevance']>0]['term'])

In [ ]:
df[df['term']=='dendritic cell migration']

In [ ]:
df[(df['per']>0.05)&(df['p_corr']<0.02)]

In [ ]:
plt.scatter(np.log(df['per']),-np.log(df['p_corr']))
plt.scatter(np.log(df[df['term']=='dendritic cell migration']['per']),-np.log(df[df['term']=='dendritic cell migration']['p_corr']))
plt.axvline(np.log(0.05),color='mediumseagreen')
plt.axhline(-np.log(0.02),color='mediumseagreen')
plt.xlabel('genes per pathway')
plt.ylabel('-log(p_corrected)')
plt.show()

In [ ]:
df[df['class']=='cellular_component']

In [ ]:

#Take only low p-value enrichments and the ones with a decent coverage
if interaction:
    
    # For interaction DEA, we need a less strict threshold
    df = df[(df['p_corr']<0.02)&(df['per']>=0.05)] ## Defaults: p_corr<0.01, per>0.05

else:

    df = df[(df['p_corr']<0.02)&(df['per']>=0.05)]

# Colorbar from all p_corr (BP, MF, CC)

import matplotlib as mpl
import textwrap

fig, ax = plt.subplots(figsize = (0.5, 2.75), dpi=600)

cmap = mpl.cm.coolwarm_r
norm = mpl.colors.Normalize(vmin = df.p_corr.min(), vmax =df.p_corr.max())

mapper = cm.ScalarMappable(norm = norm, cmap = cmap)

cbl = mpl.colorbar.ColorbarBase(ax, cmap = cmap, norm = norm, orientation = 'vertical')
plt.savefig('colorbar.pdf', bbox_inches='tight')

In [ ]:
immune_relevance_filter = True

In [ ]:
plot_fraction=True

if upregulated_degs_only==True:

    data_df = df[df['up_down']=='up'].copy()

if downregulated_degs_only==True:

    data_df = df[df['up_down']=='down'].copy()

if (upregulated_degs_only==False)&(downregulated_degs_only==False):
    
    data_df = df.copy()

if immune_relevance_filter:

    data_df = data_df[data_df['immune_relevance']>0]


data_df = data_df[(data_df['class']=='biological_process')|(data_df['class']=='cellular_component')].copy()

# data_df['score'] =  data_df['-logp_corr']/np.max(data_df['-logp_corr']) + data_df['per']/np.max(data_df['per'])

# data_df = data_df[(data_df['class']=='biological_process')].sort_values('p_corr')
# data_df = data_df.sort_values('score')[::-1]


plt.figure(figsize = (4,28))

if plot_fraction:
    
    ax = sns.barplot(data = data_df, x = 'per', y = 'term', width=0.7, palette = mapper.to_rgba(data_df.p_corr.values))

else:
    
    ax = sns.barplot(data = data_df, x = 'n_genes', y = 'term', width=0.7, palette = mapper.to_rgba(data_df.p_corr.values))



ax.set_yticklabels([textwrap.fill(e, 32) for e in data_df['term']],fontsize=10)
ax.set_ylabel('Biological process', fontsize=12)

if plot_fraction:

    ax.set_xlabel('GO term gene fraction', fontsize=12)

else:

    ax.set_xlabel('GO term genes', fontsize=12)



##### TITLE #####

### Interaction DEA
if interaction:

    if upregulated_degs_only:
    
        ax.set_title('GO: Spleen aggregated DC, interaction DEA', fontsize=12)

    else:

        ax.set_title('GO: Spleen aggregated DC, interaction DEA', fontsize=12)

        
### KO vs WT in pathogenic mice

else:

    if upregulated_degs_only:

        ax.set_title('GO: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)

    else:
        
        ax.set_title('GO: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)



#### SAVE FIG #####

### Interaction DEA
if interaction:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_BP_aggreagatedDC_interaction_patKO_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")

        else:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_BP_aggreagatedDC_interaction_patKO_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")

    else:

        if plot_fraction:
            
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_BP_aggreagatedDC_interaction_patKO_all_degs_per.pdf', dpi=600,bbox_inches = "tight")
       
        else:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_BP_aggreagatedDC_interaction_patKO_all_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")


### KO vs WT in pathogenic mice
else:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_BP_aggreagatedDC_KO_vs_WT_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")

        else:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_BP_aggreagatedDC_KO_vs_WT_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")

    else:

        if plot_fraction:
            
           plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_BP_aggreagatedDC_KO_vs_WT_all_degs_per.pdf', dpi=600,bbox_inches = "tight")

        else:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_BP_aggreagatedDC_KO_vs_WT_all_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")



plt.show()

In [ ]:
data_df[data_df['class']=='cellular_component']

In [ ]:
from itertools import combinations
import networkx as nx

# Calculate Jaccard similarity between gene sets
def jaccard_similarity(set1, set2):
    intersection = len(set(set1) & set(set2))
    union = len(set(set1) | set(set2))
    return intersection / union if union > 0 else 0

# Create a similarity matrix between terms
def create_similarity_matrix(df):
    n = len(df)
    sim_matrix = np.zeros((n, n))
    
    for i, j in combinations(range(n), 2):
        # Convert string representation of gene lists to actual sets
        genes1 = set(df.iloc[i]['study_genes'])
        genes2 = set(df.iloc[j]['study_genes'])
        
        sim = jaccard_similarity(genes1, genes2)
        sim_matrix[i, j] = sim
        sim_matrix[j, i] = sim
    
    return sim_matrix

In [ ]:
sim_matrix = create_similarity_matrix(data_df)

# Create heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(sim_matrix, 
            xticklabels=data_df['term'].str[:30],
            yticklabels=data_df['term'].str[:30],
            cmap='YlOrRd')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.title('Gene Set Overlap (Jaccard Similarity)')
plt.tight_layout()
plt.show()

In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster

# Create a network graph based on similarity threshold
G = nx.Graph()
n = len(data_df)
G.add_nodes_from(range(n))  

for i in range(n):
    for j in range(i+1, n):
        if sim_matrix[i,j] >= 0.4:
            G.add_edge(i, j, weight=sim_matrix[i,j])

# Find connected components (clusters)
clusters = list(nx.connected_components(G))

# Create visualization
plt.figure(figsize=(15, 10))

# Use spring layout for node positioning
pos = nx.spring_layout(G, k=1, iterations=50)

# Draw the network
nx.draw_networkx_nodes(G, pos, node_size=200, node_color='lightblue')
nx.draw_networkx_edges(G, pos, width=2, alpha=0.5)

# # Add labels
# labels = {i: f"{data_df['term'].iloc[i]}...\n(-logp={data_df['-logp_corr'].iloc[i]:.2f})" for i in range(len(data_df))}
# nx.draw_networkx_labels(G, pos, labels, font_size=8)

plt.title("GO Term Clusters (Similarity Threshold ≥ 0.5)")
plt.axis('off')
plt.tight_layout()
plt.show()

# Print clusters with details
print("\nDetailed Clusters (Similarity ≥ 0.75):")
print("=" * 80)
for i, cluster in enumerate(clusters):
    print(f"\nCluster {i+1}:")
    print("-" * 40)
    cluster_list = list(cluster)
    # Sort cluster members by -logp_corr
    cluster_list.sort(key=lambda x: data_df['score'].iloc[x], reverse=True)
    
    for idx in cluster_list:
        print(f"GO: {data_df['GO'].iloc[idx]}")
        print(f"Term: {data_df['term'].iloc[idx]}")
        print(f"-logp_corr: {data_df['-logp_corr'].iloc[idx]:.3f}")
        print(f" p_corr: {data_df['p_corr'].iloc[idx]:.3f}")
        print(f"score: {data_df['score'].iloc[idx]:.3f}")
        print(f"Genes: {data_df['study_genes'].iloc[idx]}")
        print("-" * 40)


# Filter out more highest p-value pathways if necessary

sig_cluster_ids = []

for i, cluster in enumerate(clusters):
    
    cluster_list = list(cluster)
    # Sort cluster members by -logp_corr
    cluster_list.sort(key=lambda x: data_df['-logp_corr'].iloc[x], reverse=True)
        
    if np.exp(-data_df['-logp_corr'].iloc[cluster_list].mean())<0.01:

        sig_cluster_ids.append(i)

sig_clusters = list(np.array(clusters)[sig_cluster_ids])

In [ ]:
for k, cluster in enumerate(clusters):
    print(k, cluster)

In [ ]:
data_df['term']

In [ ]:
data_df['term']

In [ ]:
cluster_name_dict = {}
cluster_gene_dict = {}
cluster_p_dict = {}
cluster_GO_dict = {}

for k, cluster in enumerate(clusters):

    # Let's remove Fat cell differentiation and Fibroblast GF pathways because pathways because these are likely 
    # secondary annotations due to genes' pleiotropic effects
    if ((data_df['term'].iloc[list(clusters[k])[0]] != 'fat cell differentiation')&(data_df['term'].iloc[list(clusters[k])[0]] != 'response to fibroblast growth factor')):
    
        cluster_name_dict[k]= data_df['term'].iloc[list(clusters[k])[0]]
        
        cluster_GO_dict[k] = set(data_df['GO'].iloc[list(clusters[k])])
        cluster_p_vals = data_df['p_corr'].iloc[list(clusters[k])[0]]
        cluster_genes = set(data_df['study_genes'].iloc[list(clusters[k])[0]])
        
        for j in np.arange(1,len(list(clusters[k]))):
        
            cluster_genes = cluster_genes.union(set(data_df['study_genes'].iloc[list(clusters[k])[j]]))
            cluster_p_vals = np.append(cluster_p_vals,data_df['p_corr'].iloc[list(clusters[k])[j]])
            # cluster_terms = cluster_terms.union(set(data_df['term'].iloc[list(clusters[k])[j]]))
        
        cluster_gene_dict[k]=cluster_genes
        cluster_p_dict[k]=cluster_p_vals
        

In [ ]:
cluster_name_dict

In [ ]:
#Simplify names
cluster_name_dict = {0: 'TF AP-1 Complex',
 1: 'p38MAPK Cascade',
 2: 'T Cell Differentiation and Activation',
 3: 'Interleukin-2 Production',
 5: 'Leukocyte Homeostasis',
 6: 'Non-canonical NF-kappaB Signaling',
 7: 'MHC class II Antigen Processing and Presentation ',
 8: 'Canonical NF-kappaB Signaling',
 9: 'cAMP Response',
 10: 'p53-mediated Signaling',
 11: 'Apoptotic Signaling',
 12: 'Cytoskeletal Signaling',
 13: 'Mature B Cell Differentiation',
 15: 'Interleukin-1 Production'}

In [ ]:
cluster_GO_dict

In [ ]:
cluster_p_dict

In [ ]:
cluster_gene_dict

In [ ]:
from statsmodels.stats.multitest import multipletests

## Read DESeq2 results 
def read_results(filename):
    res = pd.read_csv(filename)
    res.set_index('gene', inplace=True)
    return res

In [ ]:
# Initialize dictionary to store results across repeats
all_results = {}
eps = 1e-300  # small constant to avoid log(0)

for n_repeat in np.arange(1, 6):
    # Read results for this repeat
    results_df = read_results('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/pseudobulk/spleen/deseq2_interaction_results/spleen_aggregatedDC_deseq2_interaction_results_'+str(n_repeat)+'.csv')

    # Filter by base mean
    res = results_df[results_df.baseMean > 0.25]
    
    # Store log of raw p-values (and other statistics)
    res['log_pvalue'] = -np.log(eps + res['pvalue'])
    
    if n_repeat == 1:
        # Initialize storage for all genes we'll see
        de_res = res.copy()
        # Initialize storage for running average of log p-values
        all_genes = set(res.index)
    else:
        # Update set of all genes we've seen
        all_genes.update(res.index)
    
    # Store results for each repeat
    for idx in res.index:
        if idx not in all_results:
            all_results[idx] = {
                'log_pvalue': [],
                'baseMean': [],
                'log2FoldChange': [],
                'lfcSE': [],
                'stat': []
            }
        
        all_results[idx]['log_pvalue'].append(res.loc[idx, 'log_pvalue'])
        all_results[idx]['baseMean'].append(res.loc[idx, 'baseMean'])
        all_results[idx]['log2FoldChange'].append(res.loc[idx, 'log2FoldChange'])
        all_results[idx]['lfcSE'].append(res.loc[idx, 'lfcSE'])
        all_results[idx]['stat'].append(res.loc[idx, 'stat'])

# Create final results dataframe
final_results = []
for gene in all_genes:
    if gene in all_results:
        # Calculate mean of log p-values
        mean_log_pvalue = np.mean(all_results[gene]['log_pvalue'])
        # Convert back to p-value
        pvalue = np.exp(-mean_log_pvalue)
        
        # Calculate means of other statistics
        row = {
            'gene': gene,
            'pvalue': pvalue,
            'baseMean': np.mean(all_results[gene]['baseMean']),
            'log2FoldChange': np.mean(all_results[gene]['log2FoldChange']),
            'lfcSE': np.mean(all_results[gene]['lfcSE']),
            'stat': np.mean(all_results[gene]['stat'])
        }
        final_results.append(row)

# Convert to dataframe
final_df = pd.DataFrame(final_results)
final_df.set_index('gene', inplace=True)

# Remove NaN values before running multipletests
valid_mask = ~np.isnan(final_df['pvalue'])
valid_pvals = final_df.loc[valid_mask, 'pvalue']

# Run multipletests only on valid p-values
adj_pvals = np.full(len(final_df), np.nan)
adj_pvals[valid_mask] = multipletests(valid_pvals.values, method='fdr_bh')[1]
final_df['padj'] = adj_pvals

# # Store log-transformed versions
final_df['log_pvalue'] = -np.log(eps + final_df['pvalue'])
final_df['log_padj'] = -np.log(eps + final_df['padj'])

In [ ]:
de_res = final_df

sns.set_style("white")
plt.figure(figsize=(9,6))


plt.scatter(de_res['log2FoldChange'], np.log(de_res['log_padj']) , facecolor='dodgerblue', edgecolor='blue',linewidths=0.5, zorder=10,alpha=0.7, s=14)

plt.axhline( np.log(-np.log(0.05)) , linestyle='--', linewidth=0.8, alpha=0.7)

degs = de_res[(de_res['log_padj'] > -np.log(eps+0.05))&(np.abs(de_res['log2FoldChange'])>0.5)]

sig_logfoldchanges = degs['log2FoldChange']
sig_log_padj = degs['log_padj']
sig_var_names = list(degs.index)

top_ind = np.argsort(np.abs(sig_logfoldchanges)+3*np.log(sig_log_padj))[::-1]
top_names = np.asarray(sig_var_names)[top_ind]

for i, gene_name in enumerate(top_names[0:100]):

    # if gene_name=='Cd86':
    # print(i, gene_name)
    plt.text(sig_logfoldchanges[top_ind[i]]+0.03, np.log(sig_log_padj[top_ind[i]])+0.05, str(gene_name), fontsize=10, zorder=15)

select_names = ['Fos','Jun','Junb']
select_ind = []
[select_ind.append(np.where(np.array(sig_var_names, dtype='str')==select_names[k])[0][0]) for k in np.arange(0,len(select_names))]

for i, gene_name in enumerate(select_names):
    
    plt.text(sig_logfoldchanges[select_ind[i]]+0.03, np.log(sig_log_padj[select_ind[i]])+0.05, str(gene_name), fontsize=10, zorder=15)


plt.xlim([-8,9])
plt.ylim([0.75,5])

plt.ylabel('log(-log p)', fontsize=13)
plt.xlabel('LFC', fontsize=13)

# plt.ylim([-0.2,np.max(sig_log_padj_kowt[np.asarray(sig_var_names_kowt)!='Cd74'])+0.5])
# plt.title('Difference in response to pathogenic condition in spleen: KO vs WT')
plt.title('Diff. response to pathogenicity in spleen aggregated DCs: KO vs WT', fontsize='small')

# plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/pseudobulk/spleen/deseq2_interaction_results/volcano_kopat_vs_rest_pathogenic_spleen_DC2_interaction.pdf', dpi=600,bbox_inches = "tight")
# plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/pseudobulk/spleen/deseq2_interaction_results/volcano_kopat_vs_rest_pathogenic_spleen_CD8minusDC1_interaction.pdf', dpi=600,bbox_inches = "tight")
plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/pseudobulk/spleen/deseq2_interaction_results/volcano_kopat_vs_rest_pathogenic_spleen_aggregatedDC_interaction.pdf', dpi=600,bbox_inches = "tight")

plt.show()

top_genes=np.asarray(sig_var_names)[np.argsort(np.abs(sig_logfoldchanges)+sig_log_padj)[::-1]]

print('top_genes:',top_genes )

In [ ]:
# Read the files made with DESeq2 in R
raw_counts = pd.read_csv("/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/pseudobulk/spleen/deseq2_interaction_results/res_raw_counts.csv", index_col=0)
norm_counts = pd.read_csv("/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/pseudobulk/spleen/deseq2_interaction_results/res_normalized_counts.csv", index_col=0)
gene_info = pd.read_csv("/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/pseudobulk/spleen/deseq2_interaction_results/res_gene_info.csv", index_col=0)
sample_info = pd.read_csv("/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/pseudobulk/spleen/deseq2_interaction_results/res_sample_info.csv", index_col=0)

# Create AnnData object with raw counts as main matrix
dds_adata = ad.AnnData(
    X=raw_counts.T.values,
    var=gene_info,
    obs=sample_info
)

# Set the index names
dds_adata.var_names = raw_counts.index
dds_adata.obs_names = raw_counts.columns

# Add normalized counts in layers
dds_adata.layers['normalized'] = norm_counts.T.values

# Add size factors to obs
dds_adata.obs['size_factors'] = sample_info['sizeFactor']

# Store information about what's in the layers
dds_adata.uns['layer_description'] = {
    'X': 'raw_counts',
    'normalized': 'deseq2_normalized_counts'
}

# Verify the structure
print(dds_adata)
print("\nAvailable layers:", list(dds_adata.layers.keys()))
print("\nShape of raw counts:", dds_adata.X.shape)
print("Shape of normalized counts:", dds_adata.layers['normalized'].shape)


In [ ]:
import matplotlib.patches as patches
import matplotlib.path as mpath
from matplotlib.collections import PatchCollection

In [ ]:
len(clusters)

In [ ]:
#Define colormap for GO terms

# Get colors from tab20
tab20_colors = plt.cm.tab20.colors
tab20b_colors = plt.cm.tab20b.colors

# Combine colors from tab20 and tab20b
# combined_colors = [plt.cm.tab20b.colors[0]] + list(tab20_colors)[0:10]+ [plt.cm.tab20c.colors[6]]+ [plt.cm.tab20b.colors[11]] + list(tab20_colors)[16:20]  + [plt.cm.tab20b.colors[2]]
combined_colors = list(tab20_colors)[0:10]+  list(tab20_colors)[16:17] + list(tab20_colors)[18:19]  + [plt.cm.tab20b.colors[2]] + [plt.cm.tab20b.colors[0]] 

# Create a new ListedColormap
from matplotlib.colors import ListedColormap
custom_cmap = ListedColormap(combined_colors)

# Verify the number of colors
print(f"Number of colors in new colormap: {len(custom_cmap.colors)}")

# Optional: Visualize the colormap
fig, ax = plt.subplots(figsize=(10, 1))
gradient = np.linspace(0, 1, len(custom_cmap.colors))
gradient = np.vstack((gradient, gradient))
ax.imshow(gradient, aspect='auto', cmap=custom_cmap)
ax.set_axis_off()
plt.show()

In [ ]:
# Z-score normalization over rows

dds =dds_adata.copy()

# degs = de_res[(de_res['log_padj'] > -np.log(eps+0.05))&(np.abs(de_res['log2FoldChange'])>0.5)]

# Only upregulated degs
degs = de_res[(de_res['log_padj'] > -np.log(eps+0.05))&(de_res['log2FoldChange']>0.5)]


sig_logfoldchanges = degs['log2FoldChange']
sig_log_padj = degs['log_padj']
sig_var_names = list(degs.index)

top_genes=np.asarray(sig_var_names)[np.argsort(np.abs(sig_logfoldchanges)+sig_log_padj)[::-1]]

dds.layers['log1p'] = np.log1p(dds.layers['normalized'])
dds_sig = dds[:, sig_var_names]

log1p_counts = pd.DataFrame(dds_sig.layers['log1p'], index=dds_sig.obs.index, columns=dds_sig.var_names)

# conditions = ['wt_naive','wt_pathogenic','ko_naive','ko_pathogenic']  
conditions = ['ko_pathogenic','ko_naive','wt_pathogenic','wt_naive']  

#Construct a dataframe that will hold the mean expression levels of the filtered genes in various conditions
mean_log1p_counts = pd.DataFrame(index=conditions, columns=dds_sig.var_names, dtype='float')

for condition in conditions:
    # Get all rows that start with the current condition
    rows = [row for row in log1p_counts.index if row.startswith(condition)]
    
    # Calculate the mean of these rows
    condition_mean = np.nanmean(log1p_counts.loc[rows],axis=0)
    
    mean_log1p_counts.loc[condition] =  condition_mean.astype(float)


diff_ko = mean_log1p_counts.loc['ko_pathogenic'] - mean_log1p_counts.loc['ko_naive']
diff_wt = mean_log1p_counts.loc['wt_pathogenic'] - mean_log1p_counts.loc['wt_naive']
ddiff_ko_wt = diff_ko - diff_wt

plot_genes = top_genes

# Create a list of '+' and '-' signs based on the difference
signs_ko = {gene: '+' if d > 0 else '-' for gene, d in diff_ko[plot_genes].items()}
signs_wt = {gene: '+' if d > 0 else '-' for gene, d in diff_wt[plot_genes].items()}
signs_ko_wt = {gene: '+' if d > 0 else '-' for gene, d in ddiff_ko_wt[plot_genes].items()}


sns.set_style("white")
plt.figure(dpi=1200)

g=sns.clustermap(mean_log1p_counts[plot_genes].T,
               z_score=0,
               # standard_scale=0,  
               cmap='RdYlBu_r',
               yticklabels=True,
               xticklabels=['KO Pathogenic', 'KO Naive' ,'WT Pathogenic','WT Naive'],
               col_cluster=False,
               row_cluster=True,
               cbar_pos=None,
               dendrogram_ratio=(.3, 0.),
               tree_kws=dict(linewidths=1.0, colors=(0.2, 0.2, 0.4)),
               figsize = (8,24),
               # cbar_pos=(0.85, 0.8, 0.1, 0.05),
              )

# 2. Label modifications
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_ymajorticklabels(), fontsize=9, rotation=180)
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize=9, rotation=180)

# 3. Get the figure and axis references
fig = plt.gcf()
ax = g.ax_heatmap

# 4. Add the signs (your existing code)
reordered_genes = g.dendrogram_row.reordered_ind
for idx, gene_idx in enumerate(reordered_genes):
    gene = plot_genes[gene_idx]
    sign_ko = signs_ko[gene]
    sign_wt = signs_wt[gene]
    sign_ko_wt = signs_ko_wt[gene]

    #Sign of ko response
    rect = patches.Rectangle((0.925, idx), 0.15, 1, fill=True, facecolor='white', edgecolor='none', alpha=0.8)
    ax.add_patch(rect)
    if sign_ko=='+':
        ax.text(1.01, idx+0.5, sign_ko, ha='center', va='center', fontweight='medium', fontsize=10, rotation=90)
    else:
        ax.text(1.0, idx+0.5, sign_ko, ha='center', va='center', fontweight='medium', fontsize=10, rotation=90)

    
    #Sign of wt response
    rect = patches.Rectangle((2.925, idx), 0.15, 1, fill=True, facecolor='white', edgecolor='none', alpha=0.8)
    ax.add_patch(rect)
    if sign_wt=='+':
        ax.text(3.01, idx+0.5, sign_wt, ha='center', va='center', fontweight='medium', fontsize=10, rotation=90)
    else:
        ax.text(3.0, idx+0.5, sign_wt, ha='center', va='center', fontweight='medium', fontsize=10, rotation=90)

    #Sign of ko-wt response difference
    rect = patches.Rectangle((1.9, idx), 0.2, 1, fill=True, facecolor='white', edgecolor='none', alpha=0.9)
    ax.add_patch(rect)

    if sign_ko_wt=='+':
        ax.text(2.01, idx+0.5, sign_ko_wt, ha='center', va='center', fontweight='bold', fontsize=12, rotation=90)
    else:
        ax.text(2.0, idx+0.5, sign_ko_wt, ha='center', va='center', fontweight='bold', fontsize=12, rotation=90)


# Define colors for 12 clusters
# cluster_colors = plt.cm.tab20(np.reshape(np.linspace(0, 1, len(custom_cmap.colors)),(int(len(custom_cmap.colors)/2),2)).flatten(order='F'))
cluster_colors = custom_cmap(np.linspace(0, 1, len(custom_cmap.colors)))
# cluster_colors = plt.cm.tab20(np.linspace(0, 1, len(custom_cmap.colors)))


# Create a new axes for the cluster labels
ax_labels = fig.add_axes([
    ax.get_position().x1 + 0.05,
    ax.get_position().y0,
    0.5,
    ax.get_position().height
])

ax_labels.set_frame_on(False)
ax_labels.set_xticks([])
ax_labels.set_yticks([])

# Get the actual y-coordinates from the heatmap
heatmap_height = ax.get_position().height
y_scale = heatmap_height / len(plot_genes)
y_offset = ax.get_position().y0

# Create correct gene position mapping using scaled coordinates
gene_positions = {}
for idx, gene_idx in enumerate(reordered_genes):
    gene = plot_genes[gene_idx]
    # Scale the position to match the heatmap coordinates. Use the small deviation after -idx-1 to adjust the centre)
    scaled_pos = y_offset + (len(plot_genes) - idx - 1 + 0.5) * y_scale
    gene_positions[gene] = scaled_pos

# # Print some debugging info
# print("Y-scale:", y_scale)
# print("Y-offset:", y_offset)
# print("Sample of gene positions:", dict(list(gene_positions.items())[:5]))

def curved_path(start_pos, end_pos):
    return mpath.Path([
        (0, start_pos),
        (0.5, start_pos),
        (1.0, end_pos),
        (1.5, end_pos)
    ], [
        mpath.Path.MOVETO,
        mpath.Path.CURVE4,
        mpath.Path.CURVE4,
        mpath.Path.CURVE4
    ])

# Calculate mean positions for each cluster
cluster_mean_positions = {}
for cluster_idx, genes in cluster_gene_dict.items():
    valid_positions = [gene_positions[gene] for gene in genes if gene in gene_positions]
    if valid_positions:
        cluster_mean_positions[cluster_idx] = np.mean(valid_positions)

# Sort clusters by their mean position
sorted_clusters = sorted(cluster_mean_positions.items(), key=lambda x: x[1])

# Calculate evenly spaced positions within the same coordinate system
y_min = ax.get_position().y0
y_max = ax.get_position().y1
total_height = y_max - y_min
n_clusters = len(sorted_clusters)
spacing = total_height / (n_clusters + 1)

evenly_spaced_positions = {
    cluster_idx: y_min + spacing * (i + 1)
    for i, (cluster_idx, _) in enumerate(sorted_clusters)
}

# Draw connections for each cluster
for cluster_idx, mean_pos in sorted_clusters:
    genes = cluster_gene_dict[cluster_idx]
    cluster_genes = [gene for gene in genes if gene in gene_positions]
    cluster_GO_terms = list(cluster_GO_dict[cluster_idx])
    go_terms_formatted = '\n'.join(cluster_GO_terms)
    p_values = cluster_p_dict[cluster_idx]
    cluster_min_p = np.min(cluster_p_dict[cluster_idx])

    # print(cluster_idx)
    
    if not cluster_genes:
        continue
    
    # Use evenly spaced position for label
    label_pos = evenly_spaced_positions[cluster_idx]

    box_dist = 1.6
    # Draw curved lines to each gene
    for gene in cluster_genes:
        gene_pos = gene_positions[gene]
        path = curved_path(gene_pos, label_pos)
        patch = patches.PathPatch(
            path,
            facecolor='none',
            edgecolor=cluster_colors[np.where(np.array(sorted_clusters, dtype=int)[:,0]==cluster_idx)],
            alpha=0.8,
            linewidth=1.75
        )
        ax_labels.add_patch(patch)
    
    # Add cluster label
    bbox_props = dict(
        boxstyle="round,pad=0.3",
        fc="white",
        ec=cluster_colors[np.where(np.array(sorted_clusters, dtype=int)[:,0]==cluster_idx)],
        alpha=1
    )

    
    if (len(cluster_GO_terms)>1)&(len(cluster_GO_terms)<4):
            ax_labels.text(
                box_dist + 0.1 * cluster_idx, 
                label_pos,
                f'{cluster_name_dict[cluster_idx]}\n'
                f'{len(cluster_genes)} genes\n'
                f'min. p={cluster_min_p:.3g}\n'
                f'{go_terms_formatted}',  # Now each GO term will be on its own line
                color='black',
                va='center',
                fontsize=10,
                bbox=bbox_props,
                rotation=90
            )

    if len(cluster_GO_terms)==1:
        ax_labels.text(
            box_dist + 0.1 * cluster_idx, 
            label_pos,
            f'{cluster_name_dict[cluster_idx]}\n'
            f'{len(cluster_genes)} genes\n'
            f'p={cluster_min_p:.3g}\n'
            f'{go_terms_formatted}',  # Now each GO term will be on its own line
            color='black',
            va='center',
            fontsize=10,
            bbox=bbox_props,
            rotation=90
        )
        

    if (len(cluster_GO_terms)>3):

        ax_labels.text(
                box_dist + 0.1 * cluster_idx, 
                label_pos,
                f'{cluster_name_dict[cluster_idx]}\n'
                f'{len(cluster_genes)} genes\n'
                f'min. p={cluster_min_p:.3g}\n'
                f'{len(cluster_GO_terms)} Go terms',  # Now each GO term will be on its own line
                color='black',
                va='center',
                fontsize=10,
                bbox=bbox_props,
                rotation=90
            )
# Set proper limits
ax_labels.set_xlim(-0.1, 2.7)
ax_labels.set_ylim(y_min, y_max)

# plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/Fig5/interactionDEA_heatmap_with_GO.pdf', dpi=600,bbox_inches = "tight")

plt.show()


In [ ]:
# Without Z-score normalization

dds =dds_adata.copy()

# degs = de_res[(de_res['log_padj'] > -np.log(eps+0.05))&(np.abs(de_res['log2FoldChange'])>0.5)]

# Only upregulated degs
degs = de_res[(de_res['log_padj'] > -np.log(eps+0.05))&(de_res['log2FoldChange']>0.5)]


sig_logfoldchanges = degs['log2FoldChange']
sig_log_padj = degs['log_padj']
sig_var_names = list(degs.index)

top_genes=np.asarray(sig_var_names)[np.argsort(np.abs(sig_logfoldchanges)+sig_log_padj)[::-1]]

dds.layers['log1p'] = np.log1p(dds.layers['normalized'])
dds_sig = dds[:, sig_var_names]

log1p_counts = pd.DataFrame(dds_sig.layers['log1p'], index=dds_sig.obs.index, columns=dds_sig.var_names)

# conditions = ['wt_naive','wt_pathogenic','ko_naive','ko_pathogenic']  
conditions = ['ko_pathogenic','ko_naive','wt_pathogenic','wt_naive']  

#Construct a dataframe that will hold the mean expression levels of the filtered genes in various conditions
mean_log1p_counts = pd.DataFrame(index=conditions, columns=dds_sig.var_names, dtype='float')

for condition in conditions:
    # Get all rows that start with the current condition
    rows = [row for row in log1p_counts.index if row.startswith(condition)]
    
    # Calculate the mean of these rows
    condition_mean = np.nanmean(log1p_counts.loc[rows],axis=0)
    
    mean_log1p_counts.loc[condition] =  condition_mean.astype(float)


diff_ko = mean_log1p_counts.loc['ko_pathogenic'] - mean_log1p_counts.loc['ko_naive']
diff_wt = mean_log1p_counts.loc['wt_pathogenic'] - mean_log1p_counts.loc['wt_naive']
ddiff_ko_wt = diff_ko - diff_wt

plot_genes = top_genes

# Create a list of '+' and '-' signs based on the difference
signs_ko = {gene: '+' if d > 0 else '-' for gene, d in diff_ko[plot_genes].items()}
signs_wt = {gene: '+' if d > 0 else '-' for gene, d in diff_wt[plot_genes].items()}
signs_ko_wt = {gene: '+' if d > 0 else '-' for gene, d in ddiff_ko_wt[plot_genes].items()}


sns.set_style("white")
plt.figure(dpi=1200)

g=sns.clustermap(mean_log1p_counts[plot_genes].T,
               z_score=None,
               # standard_scale=0,  
               cmap='RdYlBu_r',
               yticklabels=True,
               xticklabels=['KO Pathogenic', 'KO Naive' ,'WT Pathogenic','WT Naive'],
               col_cluster=False,
               row_cluster=True,
               cbar_pos=None,
               dendrogram_ratio=(.3, 0.),
               tree_kws=dict(linewidths=1.0, colors=(0.2, 0.2, 0.4)),
               figsize = (8,24),
               # cbar_pos=(0.85, 0.8, 0.1, 0.05),
              )

# 2. Label modifications
g.ax_heatmap.set_yticklabels(g.ax_heatmap.get_ymajorticklabels(), fontsize=9, rotation=180)
g.ax_heatmap.set_xticklabels(g.ax_heatmap.get_xmajorticklabels(), fontsize=9, rotation=180)

# 3. Get the figure and axis references
fig = plt.gcf()
ax = g.ax_heatmap

# 4. Add the signs (your existing code)
reordered_genes = g.dendrogram_row.reordered_ind
for idx, gene_idx in enumerate(reordered_genes):
    gene = plot_genes[gene_idx]
    sign_ko = signs_ko[gene]
    sign_wt = signs_wt[gene]
    sign_ko_wt = signs_ko_wt[gene]

    #Sign of ko response
    rect = patches.Rectangle((0.925, idx), 0.15, 1, fill=True, facecolor='white', edgecolor='none', alpha=0.8)
    ax.add_patch(rect)
    if sign_ko=='+':
        ax.text(1.01, idx+0.5, sign_ko, ha='center', va='center', fontweight='medium', fontsize=10, rotation=90)
    else:
        ax.text(1.0, idx+0.5, sign_ko, ha='center', va='center', fontweight='medium', fontsize=10, rotation=90)

    
    #Sign of wt response
    rect = patches.Rectangle((2.925, idx), 0.15, 1, fill=True, facecolor='white', edgecolor='none', alpha=0.8)
    ax.add_patch(rect)
    if sign_wt=='+':
        ax.text(3.01, idx+0.5, sign_wt, ha='center', va='center', fontweight='medium', fontsize=10, rotation=90)
    else:
        ax.text(3.0, idx+0.5, sign_wt, ha='center', va='center', fontweight='medium', fontsize=10, rotation=90)

    #Sign of ko-wt response difference
    rect = patches.Rectangle((1.9, idx), 0.2, 1, fill=True, facecolor='white', edgecolor='none', alpha=0.9)
    ax.add_patch(rect)

    if sign_ko_wt=='+':
        ax.text(2.01, idx+0.5, sign_ko_wt, ha='center', va='center', fontweight='bold', fontsize=12, rotation=90)
    else:
        ax.text(2.0, idx+0.5, sign_ko_wt, ha='center', va='center', fontweight='bold', fontsize=12, rotation=90)


# Define colors for 12 clusters
# cluster_colors = plt.cm.tab20(np.reshape(np.linspace(0, 1, len(custom_cmap.colors)),(int(len(custom_cmap.colors)/2),2)).flatten(order='F'))
cluster_colors = custom_cmap(np.linspace(0, 1, len(custom_cmap.colors)))
# cluster_colors = plt.cm.tab20(np.linspace(0, 1, len(custom_cmap.colors)))


# Create a new axes for the cluster labels
ax_labels = fig.add_axes([
    ax.get_position().x1 + 0.05,
    ax.get_position().y0,
    0.5,
    ax.get_position().height
])

ax_labels.set_frame_on(False)
ax_labels.set_xticks([])
ax_labels.set_yticks([])

# Get the actual y-coordinates from the heatmap
heatmap_height = ax.get_position().height
y_scale = heatmap_height / len(plot_genes)
y_offset = ax.get_position().y0

# Create correct gene position mapping using scaled coordinates
gene_positions = {}
for idx, gene_idx in enumerate(reordered_genes):
    gene = plot_genes[gene_idx]
    # Scale the position to match the heatmap coordinates. Use the small deviation after -idx-1 to adjust the centre)
    scaled_pos = y_offset + (len(plot_genes) - idx - 1 + 0.5) * y_scale
    gene_positions[gene] = scaled_pos

# # Print some debugging info
# print("Y-scale:", y_scale)
# print("Y-offset:", y_offset)
# print("Sample of gene positions:", dict(list(gene_positions.items())[:5]))

def curved_path(start_pos, end_pos):
    return mpath.Path([
        (0, start_pos),
        (0.5, start_pos),
        (1.0, end_pos),
        (1.5, end_pos)
    ], [
        mpath.Path.MOVETO,
        mpath.Path.CURVE4,
        mpath.Path.CURVE4,
        mpath.Path.CURVE4
    ])

# Calculate mean positions for each cluster
cluster_mean_positions = {}
for cluster_idx, genes in cluster_gene_dict.items():
    valid_positions = [gene_positions[gene] for gene in genes if gene in gene_positions]
    if valid_positions:
        cluster_mean_positions[cluster_idx] = np.mean(valid_positions)

# Sort clusters by their mean position
sorted_clusters = sorted(cluster_mean_positions.items(), key=lambda x: x[1])

# Calculate evenly spaced positions within the same coordinate system
y_min = ax.get_position().y0
y_max = ax.get_position().y1
total_height = y_max - y_min
n_clusters = len(sorted_clusters)
spacing = total_height / (n_clusters + 1)

evenly_spaced_positions = {
    cluster_idx: y_min + spacing * (i + 1)
    for i, (cluster_idx, _) in enumerate(sorted_clusters)
}

# Draw connections for each cluster
for cluster_idx, mean_pos in sorted_clusters:
    genes = cluster_gene_dict[cluster_idx]
    cluster_genes = [gene for gene in genes if gene in gene_positions]
    cluster_GO_terms = list(cluster_GO_dict[cluster_idx])
    go_terms_formatted = '\n'.join(cluster_GO_terms)
    p_values = cluster_p_dict[cluster_idx]
    cluster_min_p = np.min(cluster_p_dict[cluster_idx])

    # print(cluster_idx)
    
    if not cluster_genes:
        continue
    
    # Use evenly spaced position for label
    label_pos = evenly_spaced_positions[cluster_idx]

    box_dist = 1.6
    # Draw curved lines to each gene
    for gene in cluster_genes:
        gene_pos = gene_positions[gene]
        path = curved_path(gene_pos, label_pos)
        patch = patches.PathPatch(
            path,
            facecolor='none',
            edgecolor=cluster_colors[np.where(np.array(sorted_clusters, dtype=int)[:,0]==cluster_idx)],
            alpha=0.8,
            linewidth=1.75
        )
        ax_labels.add_patch(patch)
    
    # Add cluster label
    bbox_props = dict(
        boxstyle="round,pad=0.3",
        fc="white",
        ec=cluster_colors[np.where(np.array(sorted_clusters, dtype=int)[:,0]==cluster_idx)],
        alpha=1
    )

    
    if (len(cluster_GO_terms)>1)&(len(cluster_GO_terms)<4):
            ax_labels.text(
                box_dist + 0.1 * cluster_idx, 
                label_pos,
                f'{cluster_name_dict[cluster_idx]}\n'
                f'{len(cluster_genes)} genes\n'
                f'min. p={cluster_min_p:.3g}\n'
                f'{go_terms_formatted}',  # Now each GO term will be on its own line
                color='black',
                va='center',
                fontsize=10,
                bbox=bbox_props,
                rotation=90
            )

    if len(cluster_GO_terms)==1:
        ax_labels.text(
            box_dist + 0.1 * cluster_idx, 
            label_pos,
            f'{cluster_name_dict[cluster_idx]}\n'
            f'{len(cluster_genes)} genes\n'
            f'p={cluster_min_p:.3g}\n'
            f'{go_terms_formatted}',  # Now each GO term will be on its own line
            color='black',
            va='center',
            fontsize=10,
            bbox=bbox_props,
            rotation=90
        )
        

    if (len(cluster_GO_terms)>3):

        ax_labels.text(
                box_dist + 0.1 * cluster_idx, 
                label_pos,
                f'{cluster_name_dict[cluster_idx]}\n'
                f'{len(cluster_genes)} genes\n'
                f'min. p={cluster_min_p:.3g}\n'
                f'{len(cluster_GO_terms)} Go terms',  # Now each GO term will be on its own line
                color='black',
                va='center',
                fontsize=10,
                bbox=bbox_props,
                rotation=90
            )
# Set proper limits
ax_labels.set_xlim(-0.1, 2.7)
ax_labels.set_ylim(y_min, y_max)

plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/Fig5/interactionDEA_heatmap_with_GO_noZ_score.pdf', dpi=600,bbox_inches = "tight")

plt.show()


In [ ]:


# Create a separate figure for the colorbar
fig_cbar, ax_cbar = plt.subplots(figsize=(4, 2))

# Get the normalization from the main heatmap
norm = g.ax_heatmap.collections[0].norm

# Create a colorbar
sm = plt.cm.ScalarMappable(cmap='RdYlBu_r', norm=norm)
sm.set_array([])

# Add colorbar to the new figure
cbar = plt.colorbar(sm, ax=ax_cbar, orientation='horizontal', 
                    aspect=20,  # Decrease this number to make the colorbar thicker
                    shrink=1.5)  # Adjust the length of the colorbar
ax_cbar.remove()  # Remove the axes, leaving only the colorbar

# cbar.set_ticks([-1.5, 0,  1.5])
# cbar.set_ticklabels(['%.1f' % i for i in [-1.5,  0, 1.5]])

# Adjust the figure layout
plt.tight_layout()

plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/Fig5/interactionDEA_heatmap_with_GO_cbar_horizontal_noZ.pdf', dpi=600,bbox_inches = "tight")


# Show both figures
plt.show()

In [ ]:
# antigen_terms = np.array(list([s for s in list(reactome_results['description']) if "Antigen" in s]))
# data_df[data_df['np.asarray(antigen_terms[0:5],dtype=str)]

In [ ]:
df[(df['class']=='cellular_component')]

In [ ]:
immune_relevance_filter=True

if upregulated_degs_only==True:

    data_df = df[df['up_down']=='up'].copy()

if downregulated_degs_only==True:

    data_df = df[df['up_down']=='down'].copy()

if (upregulated_degs_only==False)&(downregulated_degs_only==False):
    
    data_df = df.copy()

if immune_relevance_filter:
    data_df = data_df[data_df['immune_relevance']>0]

data_df = data_df[(data_df['class']=='cellular_component')]
data_df = data_df.sort_values('score')[::-1]

plt.figure(figsize = (4,6))

if plot_fraction:
    ax = sns.barplot(data = data_df, x = 'per', y = 'term', palette = mapper.to_rgba(data_df.p_corr.values))
    ax.set_xlabel('GO term gene fraction', fontsize=12)
    
else:
    ax = sns.barplot(data = data_df, x = 'n_genes', y = 'term', palette = mapper.to_rgba(data_df.p_corr.values))
    ax.set_xlabel('GO term genes', fontsize=12)


ax.set_yticklabels([textwrap.fill(e, 22) for e in data_df['term']],fontsize=10)
ax.set_ylabel('Cellular component', fontsize=12)
    

### Interaction DEA
if interaction:

    if upregulated_degs_only:
    
        ax.set_title('GO: Spleen aggregated DC, interaction DEA', fontsize=12)

    else:

        ax.set_title('GO: Spleen aggregated DC, interaction DEA', fontsize=12)

        
### KO vs WT in pathogenic mice

else:

    if upregulated_degs_only:

        ax.set_title('GO: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)

    else:
        
        ax.set_title('GO: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)


#### SAVE FIG #####

### Interaction DEA
if interaction:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_CC_aggreagatedDC_interaction_patKO_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")

        else:
           
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_CC_aggreagatedDC_interaction_patKO_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")

    else:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_CC_aggreagatedDC_interaction_patKO_all_degs_per.pdf', dpi=600,bbox_inches = "tight")

        else:
    
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_CC_aggreagatedDC_interaction_patKO_all_deg_ngeness.pdf', dpi=600,bbox_inches = "tight")



### KO vs WT in pathogenic mice
else:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_CC_aggreagatedDC_KO_vs_WT_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")

        else: 

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_CC_aggreagatedDC_KO_vs_WT_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")
    
    else:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_CC_aggreagatedDC_KO_vs_WT_all_degs_per.pdf', dpi=600,bbox_inches = "tight")

        else:
            
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_CC_aggreagatedDC_KO_vs_WT_all_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")




plt.show()

In [ ]:
data_df.iloc[0:10,:][['term','study_genes']]

In [ ]:

plt.figure(figsize = (4,8))
if upregulated_degs_only==True:

    data_df = df[df['up_down']=='up'].copy()

if downregulated_degs_only==True:

    data_df = df[df['up_down']=='down'].copy()

if (upregulated_degs_only==False)&(downregulated_degs_only==False):
    
    data_df = df.copy()

if immune_filter:
    data_df = data_df[data_df['immune_relevance']>0]


data_df = data_df[(data_df['class']=='molecular_function')]
data_df = data_df.sort_values('score')[::-1]


if plot_fraction:

    ax = sns.barplot(data = data_df, x = 'per', y = 'term', palette = mapper.to_rgba(data_df.p_corr.values))
    ax.set_xlabel('GO term gene fraction', fontsize=12)

else:
    
    ax = sns.barplot(data = data_df, x = 'n_genes', y = 'term', palette = mapper.to_rgba(data_df.p_corr.values))
    ax.set_xlabel('GO term genes', fontsize=12)

ax.set_yticklabels([textwrap.fill(e, 32) for e in data_df['term']], fontsize=10)
ax.set_ylabel('Molecular function', fontsize=12)


### Interaction DEA
if interaction:

    if upregulated_degs_only:
    
        ax.set_title('GO: Spleen aggregated DC, interaction DEA', fontsize=12)

    else:

        ax.set_title('GO: Spleen aggregated DC, interaction DEA', fontsize=12)

        
### KO vs WT in pathogenic mice

else:

    if upregulated_degs_only:

        ax.set_title('GO: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)

    else:
        
        ax.set_title('GO: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)

#### SAVE FIG #####

### Interaction DEA

if interaction:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")

    else:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_all_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:
            
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_all_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")


### KO vs WT in pathogenic mice
else:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")

        else:
            
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")

    else:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_all_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:    
        
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_all_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")


plt.show()

In [ ]:
df[df['term']=='transcription factor AP-1 complex']['study_genes']

### Enrichr

In [ ]:
import json
import requests

def enrichr_analysis(genes, database='KEGG_2021_Mouse'):
    # Get the enrichr link
    ENRICHR_URL = 'https://maayanlab.cloud/Enrichr/addList'
    genes_str = '\n'.join(genes)
    payload = {
        'list': (None, genes_str),
    }
    
    response = requests.post(ENRICHR_URL, files=payload)
    if not response.ok:
        raise Exception('Error analyzing gene list')
    
    data = json.loads(response.text)
    user_list_id = data['userListId']
    
    # Get the enrichment results
    ENRICHR_URL = f'https://maayanlab.cloud/Enrichr/enrich?userListId={user_list_id}&backgroundType={database}'
    response = requests.get(ENRICHR_URL)
    if not response.ok:
        raise Exception('Error getting enrichment results')
    
    data = json.loads(response.text)
    
    # Create a pandas DataFrame
    results = pd.DataFrame(data[database], 
                         columns=["rank", "description", "p_value", "z_score", "combined_score", "genes", "adjusted_p_value",'q1','q2'])

    results = results.drop(['q1','q2'], axis=1)
    
    return results.sort_values('combined_score')



In [ ]:
for k in dc_markers_up.index:
 print(dc_markers_up['Gene'][k].upper())

In [ ]:
if interaction:

    ## Aggregated DC with interaction DEA
    dc_markers = pd.read_csv('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/pseudobulk/spleen/deseq2_interaction_results/Spleen_kopat_vs_rest_interaction_degs_aggregatedDC.csv')
    dc_markers = dc_markers.rename(columns={"gene": "Gene"})

##################################################
else:
    
    ## Aggregated DC:
    dc_markers = pd.read_csv('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/pseudobulk/spleen/deseq2_pathogenic_results/Spleen_pat_kowt_degs_aggregatedDC.csv')
    dc_markers = dc_markers.rename(columns={"Unnamed: 0": "Gene"})

dc_markers_up = dc_markers[dc_markers['log2FoldChange']>0.5]
dc_markers_down = dc_markers[dc_markers['log2FoldChange']<-0.5]

In [ ]:
# Run the analysis
genes = list(dc_markers_up.Gene.values)
# Analyze same gene list with different databases
kegg_results = enrichr_analysis(genes, database='KEGG_2021_Mouse')
wiki_results = enrichr_analysis(genes, database='WikiPathways_2024_Mouse')
reactome_results = enrichr_analysis(genes, database='Reactome_Pathways_2024')
go_bp_results = enrichr_analysis(genes, database='GO_Biological_Process_2023')
go_cc_results = enrichr_analysis(genes, database='GO_Cellular_Component_2023')
go_mf_results = enrichr_analysis(genes, database='GO_Molecular_Function_2023')


# Find overlapping biological themes
kegg_pathways = list(kegg_results[kegg_results['adjusted_p_value'] < 0.05].sort_values('combined_score')[::-1]['description'])
wiki_pathways = list(wiki_results[wiki_results['adjusted_p_value'] < 0.05].sort_values('combined_score')[::-1]['description'])
reactome_pathways = list(reactome_results[reactome_results['adjusted_p_value'] < 0.05].sort_values('combined_score')[::-1]['description'])
go_bp_terms = list(go_bp_results[go_bp_results['adjusted_p_value'] < 0.05].sort_values('combined_score')[::-1]['description'])
go_cc_terms = list(go_cc_results[go_cc_results['adjusted_p_value'] < 0.05].sort_values('combined_score')[::-1]['description'])
go_mf_terms = list(go_mf_results[go_mf_results['adjusted_p_value'] < 0.05].sort_values('combined_score')[::-1]['description'])

In [ ]:
n_genes = list([])
for row in np.arange(reactome_results.shape[0]):
    
    n_genes_row = len(reactome_results.iloc[row,:]['genes'])
    n_genes = np.append(n_genes,n_genes_row)

reactome_results['n_genes'] = n_genes

In [ ]:
n_genes = list([])
for row in np.arange(kegg_results.shape[0]):
    
    n_genes_row = len(kegg_results.iloc[row,:]['genes'])
    n_genes = np.append(n_genes,n_genes_row)

kegg_results['n_genes'] = n_genes

In [ ]:
n_genes = list([])
for row in np.arange(go_bp_results.shape[0]):
    
    n_genes_row = len(go_bp_results.iloc[row,:]['genes'])
    n_genes = np.append(n_genes,n_genes_row)

go_bp_results['n_genes'] = n_genes

In [ ]:
n_genes = list([])
for row in np.arange(go_cc_results.shape[0]):
    
    n_genes_row = len(go_cc_results.iloc[row,:]['genes'])
    n_genes = np.append(n_genes,n_genes_row)

go_cc_results['n_genes'] = n_genes

In [ ]:
reactome_sigs = reactome_results[reactome_results['adjusted_p_value'] < 0.01].sort_values('adjusted_p_value')
reactome_sigs.to_csv('reactome_sigs.csv')

In [ ]:
go_bp_sigs = go_bp_results[go_bp_results['adjusted_p_value'] < 0.01].sort_values('adjusted_p_value')[::-1]
go_bp_sigs.to_csv('go_bp_sigs.csv')

In [ ]:
go_cc_sigs = go_cc_results[go_cc_results['adjusted_p_value'] < 0.05].sort_values('adjusted_p_value')[::-1]
go_cc_sigs.to_csv('go_cc_sigs.csv')

In [ ]:
kegg_sigs = kegg_results[kegg_results['adjusted_p_value'] < 0.05].sort_values('adjusted_p_value')[::-1]
kegg_sigs.to_csv('kegg_sigs.csv')

In [ ]:
wiki_sigs = wiki_results[wiki_results['adjusted_p_value'] < 0.05].sort_values('adjusted_p_value')[::-1]
wiki_sigs.to_csv('wiki_sigs.csv')

In [ ]:
plot_fraction = False

In [ ]:

import matplotlib as mpl
import textwrap

fig, ax = plt.subplots(figsize = (0.5, 2.75), dpi=600)

cmap = mpl.cm.coolwarm_r
norm = mpl.colors.Normalize(vmin = 0, vmax =0.01)

mapper = cm.ScalarMappable(norm = norm, cmap = cmap)

cbl = mpl.colorbar.ColorbarBase(ax, cmap = cmap, norm = norm, orientation = 'vertical')
plt.savefig('colorbar.pdf', bbox_inches='tight')

In [ ]:

plt.figure(figsize = (4,16))

data_df = reactome_sigs

if plot_fraction:

    ax = sns.barplot(data = data_df, x = 'n_genes', y = 'description', palette = mapper.to_rgba(data_df.adjusted_p_value.values))
    ax.set_xlabel('GO term gene fraction', fontsize=12)

else:
    
    ax = sns.barplot(data = data_df, x = 'n_genes', y = 'description', palette = mapper.to_rgba(data_df.adjusted_p_value.values))
    ax.set_xlabel('N genes', fontsize=12)

ax.set_yticklabels([textwrap.fill(e, 32) for e in data_df['description']], fontsize=8)
ax.set_ylabel('Reactome Pathways', fontsize=12)


### Interaction DEA
if interaction:

    if upregulated_degs_only:
    
        ax.set_title('Reactome: Spleen aggregated DC, interaction DEA', fontsize=12)

    else:

        ax.set_title('Reactome: Spleen aggregated DC, interaction DEA', fontsize=12)

        
### KO vs WT in pathogenic mice

else:

    if upregulated_degs_only:

        ax.set_title('Reactome: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)

    else:
        
        ax.set_title('Reactome: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)

#### SAVE FIG #####

### Interaction DEA

if interaction:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")

    else:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_all_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:
            
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_all_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")


### KO vs WT in pathogenic mice
else:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")

        else:
            
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")

    else:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_all_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:    
        
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_all_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")


plt.show()

In [ ]:
data_df.iloc[0:10][['description','genes']]

In [ ]:
# Create a similarity matrix between terms
def create_similarity_matrix_enrichr(df):
    n = len(df)
    sim_matrix = np.zeros((n, n))
    
    for i, j in combinations(range(n), 2):
        # Convert string representation of gene lists to actual sets
        genes1 = set(df.iloc[i]['genes'])
        genes2 = set(df.iloc[j]['genes'])
        
        sim = jaccard_similarity(genes1, genes2)
        sim_matrix[i, j] = sim
        sim_matrix[j, i] = sim
    
    return sim_matrix

In [ ]:
sim_matrix = create_similarity_matrix_enrichr(data_df)

# Create heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(sim_matrix, 
            xticklabels=data_df['description'].str[:30],
            yticklabels=data_df['description'].str[:30],
            cmap='YlOrRd')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.title('Gene Set Overlap (Jaccard Similarity)')
plt.tight_layout()
plt.show()

In [ ]:
data_df

In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster


# Create a network graph based on similarity threshold
G = nx.Graph()
n = len(data_df) 
for i in range(n):
    for j in range(i+1, n):
        if sim_matrix[i,j] >= 0.75:
            G.add_edge(i, j, weight=sim_matrix[i,j])

# Find connected components (clusters)
clusters = list(nx.connected_components(G))

# Create visualization
plt.figure(figsize=(15, 10))

# Use spring layout for node positioning
pos = nx.spring_layout(G, k=1, iterations=50)

# Draw the network
nx.draw_networkx_nodes(G, pos, node_size=500, node_color='lightblue')
nx.draw_networkx_edges(G, pos, width=2, alpha=0.5)

# # Add labels
# labels = {i: f"{data_df['term'].iloc[i]}...\n(-logp={data_df['-logp_corr'].iloc[i]:.2f})" for i in range(len(data_df))}
# nx.draw_networkx_labels(G, pos, labels, font_size=8)

plt.title("GO Term Clusters (Similarity Threshold ≥ 0.75)")
plt.axis('off')
plt.tight_layout()
plt.show()

# Print clusters with details
print("\nDetailed Clusters (Similarity ≥ 0.75):")
print("=" * 80)
for i, cluster in enumerate(clusters):
    print(f"\nCluster {i+1}:")
    print("-" * 40)
    cluster_list = list(cluster)
    # Sort cluster members by -logp_corr
    cluster_list.sort(key=lambda x: data_df['adjusted_p_value'].iloc[x], reverse=False)
    
    for idx in cluster_list:
        print(f"GO: {data_df['rank'].iloc[idx]}")
        print(f"Term: {data_df['description'].iloc[idx]}")
        print(f"p_corr: {data_df['adjusted_p_value'].iloc[idx]:.3f}")
        print(f"Genes: {data_df['genes'].iloc[idx]}")
        print("-" * 40)


In [ ]:
plt.figure(figsize = (4,16))

data_df = go_bp_results[go_bp_results['adjusted_p_value'] < 0.01].sort_values('adjusted_p_value')

if plot_fraction:

    ax = sns.barplot(data = data_df, x = 'n_genes', y = 'description', palette = mapper.to_rgba(data_df.adjusted_p_value.values))
    ax.set_xlabel('GO term gene fraction', fontsize=12)

else:
    
    ax = sns.barplot(data = data_df, x = 'n_genes', y = 'description', palette = mapper.to_rgba(data_df.adjusted_p_value.values))
    ax.set_xlabel('GO term genes', fontsize=12)

ax.set_yticklabels([textwrap.fill(e, 32) for e in data_df['description']], fontsize=8)
ax.set_ylabel('GO BP', fontsize=12)


### Interaction DEA
if interaction:

    if upregulated_degs_only:
    
        ax.set_title('GO: Spleen aggregated DC, interaction DEA', fontsize=12)

    else:

        ax.set_title('GO: Spleen aggregated DC, interaction DEA', fontsize=12)

        
### KO vs WT in pathogenic mice

else:

    if upregulated_degs_only:

        ax.set_title('GO: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)

    else:
        
        ax.set_title('GO: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)

#### SAVE FIG #####

### Interaction DEA

if interaction:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")

    else:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_all_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:
            
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_all_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")


### KO vs WT in pathogenic mice
else:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")

        else:
            
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")

    else:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_all_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:    
        
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_all_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")


plt.show()

In [ ]:
list(go_cc_results['description'])

In [ ]:
plt.figure(figsize = (4,16))

data_df = go_cc_results[go_cc_results['adjusted_p_value'] < 0.05].sort_values('adjusted_p_value')

if plot_fraction:

    ax = sns.barplot(data = data_df, x = 'n_genes', y = 'description', palette = mapper.to_rgba(data_df.adjusted_p_value.values))
    ax.set_xlabel('GO term gene fraction', fontsize=12)

else:
    
    ax = sns.barplot(data = data_df, x = 'n_genes', y = 'description', palette = mapper.to_rgba(data_df.adjusted_p_value.values))
    ax.set_xlabel('GO term genes', fontsize=12)

ax.set_yticklabels([textwrap.fill(e, 32) for e in data_df['description']], fontsize=8)
ax.set_ylabel('GO CC', fontsize=12)


### Interaction DEA
if interaction:

    if upregulated_degs_only:
    
        ax.set_title('GO: Spleen aggregated DC, interaction DEA', fontsize=12)

    else:

        ax.set_title('GO: Spleen aggregated DC, interaction DEA', fontsize=12)

        
### KO vs WT in pathogenic mice

else:

    if upregulated_degs_only:

        ax.set_title('GO: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)

    else:
        
        ax.set_title('GO: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)

#### SAVE FIG #####

### Interaction DEA

if interaction:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")

    else:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_all_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:
            
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_all_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")


### KO vs WT in pathogenic mice
else:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")

        else:
            
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")

    else:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_all_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:    
        
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_all_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")


plt.show()

In [ ]:
plt.figure(figsize = (4,16))

data_df = kegg_results[kegg_results['adjusted_p_value'] < 0.05].sort_values('combined_score')[::-1][0:50]

if plot_fraction:

    ax = sns.barplot(data = data_df, x = 'n_genes', y = 'description', palette = mapper.to_rgba(data_df.adjusted_p_value.values))
    ax.set_xlabel('gene fraction', fontsize=12)

else:
    
    ax = sns.barplot(data = data_df, x = 'n_genes', y = 'description', palette = mapper.to_rgba(data_df.adjusted_p_value.values))
    ax.set_xlabel('N genes', fontsize=12)

ax.set_yticklabels([textwrap.fill(e, 32) for e in data_df['description']], fontsize=8)
ax.set_ylabel('KEGG Pathways', fontsize=12)


### Interaction DEA
if interaction:

    if upregulated_degs_only:
    
        ax.set_title('KEGG: Spleen aggregated DC, interaction DEA', fontsize=12)

    else:

        ax.set_title('KEGG: Spleen aggregated DC, interaction DEA', fontsize=12)

        
### KO vs WT in pathogenic mice

else:

    if upregulated_degs_only:

        ax.set_title('KEGG: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)

    else:
        
        ax.set_title('KEGG: Spleen aggregated DC, Pathogenic KO vs WT', fontsize=12)

#### SAVE FIG #####

### Interaction DEA

if interaction:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")

    else:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_all_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:
            
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/interactionDEA/aggregatedDC/spleen_GO_MF_aggreagatedDC_interaction_patKO_all_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")


### KO vs WT in pathogenic mice
else:

    if upregulated_degs_only:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_upregulated_degs_per.pdf', dpi=600,bbox_inches = "tight")

        else:
            
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_upregulated_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")

    else:

        if plot_fraction:

            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_all_degs_per.pdf', dpi=600,bbox_inches = "tight")
        
        else:    
        
            plt.savefig('/Users/oipulk/Documents/scRNASeq/data/Eleftheria_Maranou_Mar2024/analysis/figures/GO/spleen/KO_vs_WT_in_pathogenic/aggregatedDC/spleen_GO_MF_aggreagatedDC_KO_vs_WT_all_degs_ngenes.pdf', dpi=600,bbox_inches = "tight")


plt.show()

In [ ]:
kegg_results[0:20]

In [ ]:
enrichr_results= kegg_results

if enrichr_results is not None:
    # Convert to DataFrame
    enrichr_df = pd.DataFrame(enrichr_results)
    
    if not enrichr_df.empty:
        # Sort by p-value
        enrichr_df = enrichr_df.sort_values('combined_score')[::-1]
        
        # Create visualization
        plt.figure(figsize=(4, 8))
        
        # Take top 10 pathways
        top = enrichr_df.head(30)
        
        # Create bar plot
        plt.barh(range(len(top)), -np.log10(top['adjusted_p_value']))
        
        # Customize the plot
        plt.yticks(range(len(top)), top['description'], fontsize=10)
        plt.xlabel('-log10(adj. p-value)', fontsize=12)
        plt.title('KEGG Pathway Enrichment Analysis by Enrichr', fontsize=14)
        
        # Add value labels
        for i, v in enumerate(-np.log10(top['adjusted_p_value'])):
            plt.text(v, i, f'{v:.2f}', va='center')
        
        plt.tight_layout()
        plt.show()
        
        # # Print detailed results
        # print("\nDetailed KEGG Pathway Results:")
        # print(enrichr_df[['name', 'p_value', 'term_size', 'query_size', 
        #               'intersection_size', 'effective_domain_size']].to_string())
else:
    print("No significant KEGG pathways found")


In [ ]:
# Run the analysis for downregulated genes

degs = list(dc_markers_down.Gene.values)

enrichr_results = enrichr_analysis(degs)


In [ ]:
if enrichr_results is not None:
    # Convert to DataFrame
    enrichr_df = pd.DataFrame(enrichr_results)
    
    if not enrichr_df.empty:
        # Sort by p-value
        enrichr_df = enrichr_df.sort_values('adjusted_p_value')
        
        # Create visualization
        plt.figure(figsize=(6, 10))
        
        # Take top 10 pathways
        top = enrichr_df.head(30)
        
        # Create bar plot
        plt.barh(range(len(top)), -np.log10(top['adjusted_p_value']))
        
        # Customize the plot
        plt.yticks(range(len(top)), top['description'], fontsize=10)
        plt.xlabel('-log10(adj. p-value)', fontsize=12)
        plt.title('KEGG Pathway Enrichment Analysis by Enrichr', fontsize=14)
        
        # Add value labels
        for i, v in enumerate(-np.log10(top['adjusted_p_value'])):
            plt.text(v, i, f'{v:.2f}', va='center')
        
        plt.tight_layout()
        plt.show()
        
        # # Print detailed results
        # print("\nDetailed KEGG Pathway Results:")
        # print(enrichr_df[['name', 'p_value', 'term_size', 'query_size', 
        #               'intersection_size', 'effective_domain_size']].to_string())
else:
    print("No significant KEGG pathways found")

In [ ]:
enrichr_results[enrichr_results['description']=='Th17 cell differentiation']

### gprofiler

In [ ]:
import gprofiler
from gprofiler import GProfiler
# Create a list of gene symbols
degs = list(dc_markers_up.Gene.values)

# Run KEGG pathway analysis
gp = GProfiler(return_dataframe=True)
kegg_results = gp.profile(organism='mmusculus',query=degs)

kegg_df = kegg_results[kegg_results['source']=='KEGG']

if kegg_df is not None:
    
    if not kegg_df.empty:
        # Sort by p-value
        kegg_df = kegg_df.sort_values('p_value')
        
        # Create visualization
        plt.figure(figsize=(8, 8))
        
        # Take top 10 pathways
        top_10 = kegg_df.head(30)
        
        # Create bar plot
        plt.barh(range(len(top_10)), -np.log10(top_10['p_value']))
        
        # Customize the plot
        plt.yticks(range(len(top_10)), top_10['name'], fontsize=10)
        plt.xlabel('-log10(p-value)', fontsize=12)
        plt.title('KEGG Pathways', fontsize=14)
        
        # Add value labels
        for i, v in enumerate(-np.log10(top_10['p_value'])):
            plt.text(v, i, f'{v:.2f}', va='center')
        
        plt.tight_layout()
        plt.show()
        
        # Print detailed results
        # print("\nDetailed KEGG Pathway Results:")
        # print(kegg_df[['name', 'p_value', 'term_size', 'query_size', 
        #               'intersection_size', 'effective_domain_size']].to_string())
else:
    print("No significant KEGG pathways found")


kegg_df = kegg_results[kegg_results['source']=='GO:BP']

if kegg_results is not None:
    
    if not kegg_df.empty:
        # Sort by p-value
        kegg_df = kegg_df.sort_values('p_value')
        
        # Create visualization
        plt.figure(figsize=(8, 8))
        
        # Take top 10 pathways
        top_10 = kegg_df.head(30)
        
        # Create bar plot
        plt.barh(range(len(top_10)), -np.log10(top_10['p_value']))
        
        # Customize the plot
        plt.yticks(range(len(top_10)), top_10['name'], fontsize=10)
        plt.xlabel('-log10(p-value)', fontsize=12)
        plt.title('GO: BP', fontsize=14)
        
        # Add value labels
        for i, v in enumerate(-np.log10(top_10['p_value'])):
            plt.text(v, i, f'{v:.2f}', va='center')
        
        plt.tight_layout()
        plt.show()
        
        # Print detailed results
        # print("\nDetailed KEGG Pathway Results:")
        # print(kegg_df[['name', 'p_value', 'term_size', 'query_size', 
        #               'intersection_size', 'effective_domain_size']].to_string())
else:
    print("No significant KEGG pathways found")

kegg_df = kegg_results[kegg_results['source']=='GO:CC']

if kegg_results is not None:
    
    if not kegg_df.empty:
        # Sort by p-value
        kegg_df = kegg_df.sort_values('p_value')
        
        # Create visualization
        plt.figure(figsize=(8, 8))
        
        # Take top 10 pathways
        top_10 = kegg_df.head(30)
        
        # Create bar plot
        plt.barh(range(len(top_10)), -np.log10(top_10['p_value']))
        
        # Customize the plot
        plt.yticks(range(len(top_10)), top_10['name'], fontsize=10)
        plt.xlabel('-log10(p-value)', fontsize=12)
        plt.title('GO: CC', fontsize=14)
        
        # Add value labels
        for i, v in enumerate(-np.log10(top_10['p_value'])):
            plt.text(v, i, f'{v:.2f}', va='center')
        
        plt.tight_layout()
        plt.show()
        
        # Print detailed results
        # print("\nDetailed KEGG Pathway Results:")
        # print(kegg_df[['name', 'p_value', 'term_size', 'query_size', 
        #               'intersection_size', 'effective_domain_size']].to_string())
else:
    print("No significant KEGG pathways found")


In [ ]:
def create_enrichment_dotplot(df, n_terms=20):
    """
    Create a dot plot of GO enrichment results with different metrics
    
    Parameters:
    -----------
    df : pd.DataFrame
        Enrichment results
    """
    # Get top terms
    plot_df = df.nsmallest(n_terms, 'p_corr').copy()

    plot_df = plot_df.sort_values('p_corr', ascending=False)
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot scatter using percentage for color
    scatter = ax.scatter(x=-np.log10(plot_df['p_corr']),
                        y=plot_df['term'],
                        s=plot_df['n_go'].astype('float'),  # Size represents number of genes
                        c=plot_df['per'],  # Color represents percentage
                        cmap='winter_r')
    
    # Customize plot
    ax.set_xlabel('-log10(Adjusted p-value)')
    ax.set_ylabel('GO Term')
    
    # Add colorbar
    plt.colorbar(scatter, label='Percentage of genes')
    
    return fig


def create_go_network(df, go_relationships, n_terms=15):
    """
    Create a network visualization of GO terms
    """
    import networkx as nx
    from adjustText import adjust_text
    
    # Create network
    G = nx.Graph()
    
    # Get top terms
    top_terms = df.nsmallest(n_terms, 'p_corr')
    
    # Add nodes
    for _, row in top_terms.iterrows():
        G.add_node(row['term'], pvalue=row['p_corr'], size=row['n_genes'])
    
    # Add edges based on GO relationships
    for term1 in G.nodes():
        for term2 in G.nodes():
            if term1 != term2 and (term1, term2) in go_relationships:
                G.add_edge(term1, term2)
    
    # Create plot
    fig, ax = plt.subplots(figsize=(12, 12))
    
    # Calculate node colors and sizes
    node_colors = [-np.log10(G.nodes[node]['pvalue']) for node in G.nodes()]
    node_sizes = [G.nodes[node]['size'] for node in G.nodes()]
    
    # Draw network
    pos = nx.spring_layout(G, k=1.5, iterations=50)
    nx.draw_networkx_nodes(G, pos, 
                          node_color=node_colors,
                          node_size=50*np.sqrt(node_sizes),
                          cmap='YlOrRd',
                          alpha=0.7)
    nx.draw_networkx_edges(G, pos, alpha=0.2)
    
    # Add labels with adjustText
    texts = []
    for node, (x, y) in pos.items():
        texts.append(plt.text(x, y, node, fontsize=8))
    
    adjust_text(texts, arrowprops=dict(arrowstyle='-', color='gray', alpha=0.5))
    
    plt.title('GO Term Network')
    plt.axis('off')
    
    return fig



In [ ]:
data_df = df[df['up_down']=='up']
data_df = data_df[(data_df['class']=='biological_process')].sort_values('p_corr')
# data_df = data_df[(data_df['class']=='molecular_function')].sort_values('p_corr')


# Now let's create the visualizations
# First, the dot plot
fig1 = create_enrichment_dotplot(data_df, n_terms=30)
plt.title('GO Term Enrichment Analysis')
plt.tight_layout()
plt.show()

# Create simple GO relationships based on p-values
go_relationships = {}
sorted_terms = data_df.sort_values('p_corr').head(20)
for i, row1 in sorted_terms.iterrows():
    for j, row2 in sorted_terms.iterrows():
        if i != j:
            # Create relationship if terms are significantly enriched
            if row1['p_corr'] < 0.01 and row2['p_corr'] < 0.0001:
                go_relationships[(row1['term'], row2['term'])] = 1

# Create network visualization
fig2 = create_go_network(data_df, go_relationships, n_terms=15)
plt.show()


In [ ]:
def create_enrichment_dotplot(df, n_terms=20):
    """
    Create side-by-side dot plots for up and down regulated genes
    
    Parameters:
    -----------
    df : pd.DataFrame
        Enrichment results with 'up_down' column indicating 'up' or 'down'
    n_terms : int
        Number of terms to show for each regulation direction
    """
    # Create figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
    
    # Process upregulated genes
    df_up = df[df['up_down'] == 'up']
    plot_df_up = df_up.nsmallest(n_terms, 'p_corr').copy()
    plot_df_up = plot_df_up.sort_values('p_corr', ascending=False)
    
    # Process downregulated genes
    df_down = df[df['up_down'] == 'down']
    plot_df_down = df_down.nsmallest(n_terms, 'p_corr').copy()
    plot_df_down = plot_df_down.sort_values('p_corr', ascending=False)
    
    # Plot upregulated genes
    plt.sca(ax1)
    scatter1 = ax1.scatter(x=-np.log10(plot_df_up['p_corr']),
                          y=plot_df_up['term'],
                          s=1000 * plot_df_up['per'] / (plot_df_up['per'].max()),
                          c=plot_df_up['n_go'],
                          cmap='winter')
    ax1.set_xlabel('-log10(Adjusted p-value)')
    ax1.set_ylabel('GO Term')
    ax1.set_title('Upregulated DEGs')
    plt.colorbar(scatter1, ax=ax1, label='Number of genes in GO term')
    
    # Plot downregulated genes
    plt.sca(ax2)
    scatter2 = ax2.scatter(x=-np.log10(plot_df_down['p_corr']),
                          y=plot_df_down['term'],
                          s=1000 * plot_df_down['per'] / (plot_df_down['per'].max()),
                          c=plot_df_down['n_go'],
                          cmap='winter')
    ax2.set_xlabel('-log10(Adjusted p-value)')
    ax2.set_ylabel('GO Term')
    ax2.set_title('Downregulated DEGs')
    plt.colorbar(scatter2, ax=ax2, label='Number of genes in GO term')
    
    # Adjust layout
    plt.tight_layout()
    
    return fig

# Use the function with your concatenated DataFrame
fig = create_enrichment_dotplot(df, n_terms=20)
plt.show()
